# 🧬 シングルセル解析
このノートブックでは、`scanpy` を用いて、ヒト末梢血単核細胞（PBMC）から得られた scRNA-seq データ（PBMC3K）を処理し、クラスタリングと PanglaoDB マーカーによる細胞型推定を行う。

---
## 📚 手順概要

1. データ読み込み（PBMC3k）
2. 品質管理（QC）と前処理
3. 正規化とスケーリング
4. 次元削減とクラスタリング
5. クラスタごとのマーカー遺伝子抽出
6. PanglaoDBのマーカー遺伝子を読み込み
7. クラスタごとに細胞型を推定（PanglaoDBと照合）
8. 細胞型ラベルを各細胞に割り当てる
9. 細胞型ラベルでUMAPを描画

## 🚀 ライブラリのインストール（初回のみ）

In [ ]:
!pip install scanpy
!pip install igraph
!pip install leidenalg

関連ライブラリを読み込む

In [ ]:
import scanpy as sc
import pandas as pd

## 📥 1. データ読み込み（PBMC3k）

In [ ]:
adata = sc.datasets.pbmc3k()

In [ ]:
adata

2700細胞 x 32738遺伝子のAnnDataが作成されました。

各細胞の全リードに対して占める割合が最も高い遺伝子を表示します。

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, )

ここからどんどんフィルタリングします。

## 🔬 2. 品質管理（QC）と前処理
発現遺伝子数が200未満の細胞、発現細胞数が3未満の遺伝子をフィルタします。

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
adata

発現細胞数が3未満の19,024遺伝子がフィルタされ、2700細胞 x 13714遺伝子のAnnDataになりました。

次に、品質評価に重要なミトコンドリア遺伝子の発現量を計測します。 ミトコンドリアの発現割合が高い細胞は死細胞とみなされます。 (Lun, McCarthy & Marioni, 2017)

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

異常な細胞（ミトコンドリア高い、遺伝子数多すぎ）を除去）
ミトコンドリア由来の遺伝子が多すぎる細胞(5%以上)、またはtotal countsの値が多すぎる細胞(2500以上)を削除します。

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]
adata

2638細胞 x 13714遺伝子のAnnDataになりました。

## 🧪 3. 正規化とスケーリング
各細胞の総カウントをそろえて10000に正規化

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

対数変換（log1p = log(x+1)）

In [ ]:
sc.pp.log1p(adata)

変動の大きい遺伝子（高変動遺伝子）の選択

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata = adata[:, adata.var.highly_variable]

スケーリング（平均0・分散1に標準化）

In [ ]:
sc.pp.scale(adata, max_value=10)

## 🌐 4. 次元削減とクラスタリング
主成分分析（PCA）

In [ ]:
sc.tl.pca(adata)

近傍グラフの計算を行います。これは細胞間の距離（類似性や関連性）を数値化し、細胞間の関係をグラフ構造で表現することを目的としています。

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

UMAPによる2次元可視化

In [ ]:
sc.tl.umap(adata)

近傍グラフのクラスタリングを行います。

In [ ]:
sc.tl.leiden(adata, resolution=0.5)  # louvainでも可

クラスタごとに色を付けてUMAPを表示

In [ ]:
sc.pl.umap(adata, color='leiden', title='Leiden Clustering')

## 🧬 5. クラスタごとのマーカー遺伝子抽出

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='leiden', method='wilcoxon')

各クラスタのマーカー遺伝子（上位20）を可視化

In [ ]:
sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False)

## 📥 6. PanglaoDBのマーカー遺伝子を読み込み

In [ ]:
import pandas as pd
panglao = pd.read_csv("https://github.com/iwata97/bioinfo/raw/refs/heads/main/PanglaoDB_markers_27_Mar_2020.tsv", sep="\t")

ヒトのマーカーのみ抽出

In [ ]:
panglao = panglao[panglao["species"] == "Hs"]
panglao.head()

## 🔍 7. クラスタごとに細胞型を推定（PanglaoDBと照合）

In [ ]:
cluster_to_celltype = {}

for cluster in adata.obs['leiden'].cat.categories:
    # 各クラスタの上位20マーカー遺伝子を取得
    genes = adata.uns['rank_genes_groups']['names'][cluster][:20]
    # PanglaoDBで一致する細胞型を検索
    matched = panglao[panglao['official gene symbol'].isin(genes)]
    if not matched.empty:
        predicted_type = matched['cell type'].value_counts().idxmax()
    else:
        predicted_type = 'Unknown'
    cluster_to_celltype[cluster] = predicted_type

結果表示

In [ ]:
cluster_to_celltype

## 🏷 8. 細胞型ラベルを各細胞に割り当てる

In [ ]:
adata.obs['celltype'] = adata.obs['leiden'].map(cluster_to_celltype)

## 🎨 9. 細胞型ラベルでUMAPを描画

In [ ]:
sc.pl.umap(adata, color='celltype', legend_loc='on data', title='Predicted Cell Types from PanglaoDB')

参考ページ
https://singlecellanalysistutorial.readthedocs.io/en/latest/notebooks/Scanpy_PBMC.html?utm_source=chatgpt.com
https://qiita.com/nikera/items/45cc4d6491182d1e0aab